In [215]:
%load_ext autoreload
%autoreload 2

import cvxpy as cp
import numpy as np

import torch
from torch.nn.parameter import Parameter
import torch.optim as optim

import matplotlib.pyplot as plt
from tqdm import tqdm
from numpy import genfromtxt

%matplotlib inline

from controllers import DeePC, DDeePC
plt.rcParams["figure.figsize"] = (15,5)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [216]:
ud = genfromtxt('recht_ud.csv', delimiter=',')
yd = genfromtxt('recht_yd.csv', delimiter=',')

In [217]:
A = np.array([[1.01, 0.01, 0.00], # A - State-space matrix
              [0.01, 1.01, 0.01], 
              [0.00, 0.01, 1.01]]) 
B = np.eye(3) # B - State-space matrix
C = np.eye(3) # C - State-space matrix
D = np.zeros([3,3]) # D - State-space matrix
n = A.shape[0] # n = number of states
m = B.shape[1] # m = number of inputs
p = C.shape[0] # p = number of output
q = m+p # q = number of i/o variables

Tini = 4                                   # Past time horizon                                           
Tf = 10                                     # Future time horizon         
T = (m+1)*(Tini + Tf + n) - 1    
noise_std = 0.1                       

In [218]:
# yd += np.random.rand(T,p)*noise_std
u_ini = ud[:Tini,:].reshape((Tini*m,))
y_ini = yd[:Tini,:].reshape((Tini*p,))
y_constraints = np.kron(np.ones(Tf), np.array([10,10,10]))
u_constraonts = np.kron(np.ones(Tf), np.array([2,2,2]))

In [219]:
q = torch.ones(3)*10
r = torch.ones(3)*0.1
lam_y = torch.Tensor([0.7])

expert = DDeePC(
    ud=ud, yd=yd, N=Tf, Tini=Tini, T=T, p=3, m=3,
    y_constraints=y_constraints, u_constraints=u_constraonts,
    stochastic=False, linear=True, q=q, r=r
)

learner = DDeePC(
    ud=ud, yd=yd, N=Tf, Tini=Tini, T=T, p=3, m=3,
    y_constraints=y_constraints, u_constraints=u_constraonts,
    stochastic=False, linear=True
)

In [220]:
for param in expert.parameters():
    print(param)

for param in learner.parameters():
    print(param)

Parameter containing:
tensor([0.8996, 1.0398, 0.9432], requires_grad=True)
Parameter containing:
tensor([1.2128, 0.9633, 1.1558], requires_grad=True)


In [221]:
optim = torch.optim.RMSprop(learner.parameters(), lr=1e-2)
pbar = tqdm(range(200), ncols=120)
criterion = torch.nn.MSELoss()

for i in pbar:
    ref = torch.Tensor(np.random.uniform(size=(3,), low=-2.0, high=2.0))
    ref = torch.kron(torch.ones(Tf), ref)
    # ref += torch.Tensor(np.random.randn(Tf*p,)*noise_std)

    u_pred, _ = learner(ref=ref, u_ini=torch.Tensor(u_ini), y_ini=torch.Tensor(y_ini))
    u, _ = expert(ref=ref, u_ini=torch.Tensor(u_ini), y_ini=torch.Tensor(y_ini))

    loss = criterion(input=u_pred, target=u)
    optim.zero_grad()
    loss.backward()
    optim.step()
    
    pbar.set_description(f'Loss = {loss.item():.10f}')

Loss = 0.0000000001: 100%|████████████████████████████████████████████████████████████| 200/200 [00:10<00:00, 18.58it/s]


In [222]:
for param in learner.parameters():
    print(param)

Parameter containing:
tensor([1.6639, 1.6486, 1.6586], requires_grad=True)
Parameter containing:
tensor([0.0166, 0.0165, 0.0166], requires_grad=True)
